In [ ]:
# 출처 : https://medium.com/@nurlan.imanov/customer-segmentation-using-rfm-analysis-d2df1dfa2f9f

In [1]:
# DB접속
import os
from os import listdir
import cx_Oracle as oci
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

# 그래프
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)


In [2]:
# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_info_MMBSHIP.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

### 1.데이터 불러오기

In [6]:
###############
# 1.데이터 불러오기 : 최근 25개월간, 방문회원  
#####

# query run
query1 = '''
        select * from (
        /*통합멤버십 : 쿠폰 운영현황(이벤트성)*/
        select M.CPN_CD   AS 쿠폰코드
        , (case when M.USE_COOPCO_CD=7060 then 'ILP' when M.USE_COOPCO_CD=7050 then 'CYK' else 'PBS' end )AS 관계사
        , (select code_nm from JT_CODE	where code_grp_id = 'CM041' and M.OFFR_TYP_CD=code)   AS 쿠폰유형
        , M.CPN_NM        AS 쿠폰이름
        , M.APCL_STR_DT   AS 시작일자
        , substr(M.APCL_STR_DT,1,6)   AS 시작일자_구분
        , M.APCL_END_DT   AS 종료일자
        , (case when M.APCL_END_DT > to_char(sysdate-1, 'YYYYMMDD') then '진행중' else '종료' end) as "진행여부"
        , A.발급인원 as "발급인원(명)"
        , B.사용인원 as "사용인원(명)"
        , round(B.사용인원/A.발급인원,4)*100 as "사용율(%)" 
        , B.총결제금액 as "총결제금액(원)"
        , B.할인금액 as "할인금액(원)"
        , B.사용일자 as "최종사용일"

        from mmbship.TPM_CPN M

          --A. 다운로드 인원*
          left join (select cpn_cd
          , count(*) as 발급인원
          from mmbship.TPM_CPN_ISSU
          group by cpn_cd
          ) A
          on M.cpn_cd = A.cpn_cd

          --B. 사용 및 할인 금액
          left join (select I.cpn_cd
          , count(I.unfy_mmb_no) as 사용인원
           , sum(P.tot_sel_amt) as 총결제금액
           , sum(U.DC_AMT) as 할인금액
           , max(U.apv_dt) as 사용일자
           from mmbship.TPM_CPN_use U, mmbship.TPM_CPN_ISSU I, tp_trsc_ptcl P
           where I.CPN_ISSU_CD = U.CPN_ISSU_CD
           and P.apv_no = U.apv_no
           and I.cpn_st_cd ='2'
           and U.can_apv_dt is null
           group by I.cpn_cd
          ) B
          on M.cpn_cd = B.cpn_cd

        where M.USE_YN ='Y'
        --AND M.CUR_ISSU_QNT IS NOT NULL
        and M.APCL_END_DT not in (29991231, 99991231, 20170807, 20171203, 20171209)
        ORDER BY M.APCL_STR_DT
        ) where 쿠폰이름 like '%발렌타인%' 
        '''
df1_A = pd.read_sql(query1, con = conn )

In [8]:
df1_A = df1_A[df1_A['관계사']!='PBS']

In [9]:
df1_A

,쿠폰코드,관계사,쿠폰유형,쿠폰이름,시작일자,시작일자_구분,종료일자,진행여부,발급인원(명),사용인원(명),사용율(%),총결제금액(원),할인금액(원),최종사용일
0,CP190212002,ILP,할인권,[Maeil Do!] 발렌타인데이 이벤트 쿠폰,20190213,201902,20190217,종료,9447,27.0,0.29,1353000.0,270000.0,20190217
1,CP190212001,CYK,할인권,[Maeil Do!] 발렌타인데이 이벤트 쿠폰,20190213,201902,20190217,종료,9447,59.0,0.62,2584000.0,590000.0,20190217
2,CP200211001,CYK,할인권,[Maeil Do!] 발렌타인데이 이벤트 쿠폰,20200212,202002,20200216,종료,155,30.0,19.35,1805900.0,600000.0,20200216
3,CP200211002,ILP,할인권,[Maeil Do!] 발렌타인데이 이벤트 쿠폰,20200212,202002,20200216,종료,138,22.0,15.94,1490500.0,440000.0,20200216
4,CP210205002,ILP,할인권,[Maeil Do] 2021 설&발렌타인 할인 쿠폰,20210208,202102,20210214,종료,202,11.0,5.45,682000.0,220000.0,20210214
6,CP210205001,CYK,할인권,[Maeil Do] 2021 설&발렌타인 할인 쿠폰,20210208,202102,20210214,종료,203,19.0,9.36,1125500.0,380000.0,20210214
7,CP220210002,ILP,할인권,[Maeil Do!] 2022 발렌타인데이 이벤트 쿠폰,20220211,202202,20220220,종료,265,19.0,7.17,1442500.0,380000.0,20220220
8,CP220210001,CYK,할인권,[Maeil Do!] 2022 발렌타인데이 이벤트 쿠폰,20220211,202202,20220220,종료,330,130.0,39.39,8179400.0,2600000.0,20220220


In [10]:
pd.pivot_table(df1_A, values='사용인원(명)', index=['시작일자_구분'],columns=['관계사'],  aggfunc='sum')

관계사,CYK,ILP
시작일자_구분,,
201902,59.0,27.0
202002,30.0,22.0
202102,19.0,11.0
202202,130.0,19.0


### 2.쿠폰 발급자 성향 

In [36]:
###############
# 1.데이터 불러오기 : 최근 25개월간, 방문회원  
#####

# query run
query2 = '''
        select A.unfy_mmb_no
        , to_char(A.reg_DTM, 'YYYY-MM-DD') as USE_DT
        , to_char(B.reg_DTM, 'YYYY-MM-DD') as 쿠폰사용일
        , (case when a.cpn_st_cd = '2' then '사용' else '미사용' end) as 쿠폰사용
        , I.mmb_id
        , to_char(sysdate, 'YYYY')-substr(I.btdy,1,4) as 연령
        , (case when to_char(sysdate, 'YYYY')-substr(I.btdy,1,4) between 20 and 49
               then substr(to_char(sysdate, 'YYYY')-substr(I.btdy,1,4),1,1) || '0대'
               when to_char(sysdate, 'YYYY')-substr(I.btdy,1,4) between 50 and 89
               then '50대 이상' else null end)  as 연령대
        , (case when I.GNDR_DV_CD in( '2','4') then 'F' else 'M' end) as 성별
        , substr(trim(D.ZIP_BASE_ADDR),1,2) as 지역
        , (select P.tot_sel_amt from tp_trsc_ptcl P where P.apv_no = B.apv_no) as 결제금액
        , B.DC_AMT as 할인금액

        , B.stor_no as STOR_CD
        , (select STOR_NM from TC_AFLTSTOR where B.stor_no = stor_no) as 사용매장_구분

        from mmbship.TPM_CPN_use B, mmbship.TPM_CPN_ISSU A, mmbship.tm_mmb_info I, mmbship.TM_MMB_ADDR D
        where A.CPN_ISSU_CD = B.CPN_ISSU_CD(+)
        and B.can_apv_dt is null
        and A.unfy_mmb_no = I.unfy_mmb_no(+)
        and A.unfy_mmb_no = D.unfy_mmb_no(+)
        and A.cpn_cd in( 'CP220210002' )
        order by B.reg_DTM
        '''
df2 = pd.read_sql(query2, con = conn )
df2['UNFY_MMB_NO'] = df2['UNFY_MMB_NO'].astype(object)

In [147]:
###############
# 1.데이터 불러오기 : 최근 25개월간, 방문회원  
#####

# query run
query2 = '''
        with user_info as 
            (select A.unfy_mmb_no
             from  mmbship.TPM_CPN_ISSU A
             where A.cpn_cd in( 'CP220210002' )
             )
        select I.unfy_mmb_no, to_char(H.reg_dtm , 'YYYY-MM') as REG_DT, count(*) as CNT
        from mmbship.tm_lgn_hist H, user_info I 
        where I.unfy_mmb_no = H.unfy_mmb_no
        and H.lgn_path  in ( '371', '372')
        and H.reg_dtm between sysdate-365 and sysdate
        group by I.unfy_mmb_no, to_char(H.reg_dtm , 'YYYY-MM')
        '''
df2_lgn = pd.read_sql(query2, con = conn )
df2_lgn['UNFY_MMB_NO'] = df2_lgn['UNFY_MMB_NO'].astype(object)

In [148]:
df2_lgn_avg = pd.DataFrame(df2_lgn.groupby('UNFY_MMB_NO')['CNT'].mean()) 

In [149]:
df2_user = pd.merge(df2, df2_lgn_avg, on =['UNFY_MMB_NO'], how ='left')

In [150]:
pd.DataFrame(df2_user.groupby('쿠폰사용')['CNT'].mean()) 

,CNT
쿠폰사용,
미사용,5.744561
사용,6.773408


In [37]:
# query run
query3 = '''
        /* 기프트카드 사용이력*/
        select M.unfy_mmb_no
        , sum(A.use_amt) as use_amt
        from mmbship.tpr_crd_mstr M ,  mmbship.tpr_crd_actv_amt_use_hist A
        where M.crd_id = A.crd_id
        and A.coopco_cd = 7060
        and A.use_dt between '20210201' and '20220211'
        and  M.unfy_mmb_no is not null
        group by M.unfy_mmb_no
        '''
df3 = pd.read_sql(query3, con = conn )
df3['UNFY_MMB_NO'] = df3['UNFY_MMB_NO'].astype(object)

In [38]:
# query run
query4 = '''
        /* 기프트카드 유형별 잔액 */
        select M.unfy_mmb_no
        , (select C.usage_coopco_Cd  from  tpr_crd_kind_mstr C where C.crd_kind = M.crd_kind) as usage_coopco_Cd
        , (select C.crd_kind_nm  from  tpr_crd_kind_mstr C where C.crd_kind = M.crd_kind) as crd_kind
        , M.actv_amt - M.use_amt as remain_amt
        from  mmbship.tpr_crd_mstr M ,  mmbship.tpr_crd_actv_amt_use_hist A
        where M.crd_id = A.crd_id
        and A.coopco_cd = 7060
        and A.use_dt between '20210201' and '20220211'
        and  M.unfy_mmb_no is not null
        '''
df4 = pd.read_sql(query4, con = conn )
df4['UNFY_MMB_NO'] = df4['UNFY_MMB_NO'].astype(object)

In [39]:
df_a = pd.merge(df2, df3,  on =['UNFY_MMB_NO'], how ='left')
# df_a = pd.merge(df_a, df4, on =['UNFY_MMB_NO'], how ='left')

In [40]:
#####

s_dt1= (datetime.today()- timedelta(0)).strftime('%Y%m%d')

excel_name = s_dt1+ '_promotion_data_CJ.xlsx'
writer = pd.ExcelWriter(excel_name)
df_a.to_excel(writer, 'Sheet1')
df4.to_excel(writer, 'Sheet2')
writer.save()
writer.close()


In [190]:
df_a['USE_AMT'].mean(), df_a[df_a['쿠폰사용'] =='사용']['USE_AMT'].mean()

(125116.06060606061, 207511.11111111112)

In [195]:
df_a['USE_AMT'].mean(), df_a[df_a['쿠폰사용'] =='사용']['USE_AMT'].mean()

(194506.61971830987, 235039.0579710145)

In [ ]:
235039.057971

In [197]:
# pd.DataFrame(df_a[df_a['USE_AMT']>=0].groupby('CRD_KIND')['UNFY_MMB_NO'].count())

### 3.LMS 발송자 분석

In [36]:
# query run
query5 = '''
        select member_id as unfy_mmb_no, 'Y' as send_yn
        from tms_camp_send_list_02@dblink_tms
        where channel_type = 'SM'
        and post_id = '2022021000008'
        '''
df5 = pd.read_sql(query5, con = conn )
df5['UNFY_MMB_NO'] = df5['UNFY_MMB_NO'].astype(int)
df5['UNFY_MMB_NO'] = df5['UNFY_MMB_NO'].astype(object)

In [37]:
df_b = pd.merge(df2, df5, on ='UNFY_MMB_NO', how ='left')

In [38]:
df_b.tail(2)

,UNFY_MMB_NO,USE_DT,쿠폰사용일,쿠폰사용,MMB_ID,연령,연령대,성별,지역,결제금액,할인금액,STOR_CD,사용매장_구분,SEND_YN
263,2624711,2022-02-16,None,미사용,jungeun99,41,40대,F,서울,NaN,NaN,None,None,Y
264,4778486,2022-02-11,None,미사용,gong909,49,40대,F,서울,NaN,NaN,None,None,Y


In [39]:
df_b.groupby(['쿠폰사용', 'SEND_YN']).count()

,,UNFY_MMB_NO,USE_DT,쿠폰사용일,MMB_ID,연령,연령대,성별,지역,결제금액,할인금액,STOR_CD,사용매장_구분
쿠폰사용,SEND_YN,,,,,,,,,,,,
미사용,Y,164,164,0,164,164,164,164,87,0,0,0,0
사용,Y,16,16,16,16,16,16,16,5,16,16,16,16


In [109]:
df_b.groupby(['쿠폰사용'])['UNFY_MMB_NO'].count()

쿠폰사용
미사용    200
사용     130
Name: UNFY_MMB_NO, dtype: int64

In [110]:
df_b.groupby(['쿠폰사용', 'SEND_YN'])['UNFY_MMB_NO'].count() / df_b.groupby(['쿠폰사용'])['UNFY_MMB_NO'].count()

쿠폰사용  SEND_YN
미사용   Y          0.635000
사용    Y          0.646154
Name: UNFY_MMB_NO, dtype: float64

### 4.과거 쿠폰이용 고객

In [143]:
# query run
query6 = '''
        select unfy_mmb_no, count(*) as coupon_CNT
        from (
        select A.unfy_mmb_no
        , B.stor_no as STOR_CD
        , (select STOR_NM from TC_AFLTSTOR where B.stor_no = stor_no) as 사용매장_구분
        from mmbship.TPM_CPN_use B, mmbship.TPM_CPN_ISSU A
        where A.CPN_ISSU_CD = B.CPN_ISSU_CD
        and B.coopco_cd = '7050' 
        and a.cpn_st_cd = '2'
        and A.unfy_mmb_no not in ('4922343' )
        and A.reg_DTM between sysdate-365 and sysdate-35)
        group by unfy_mmb_no
        order by count(*) desc
        '''
df6 = pd.read_sql(query6, con = conn )
df6['UNFY_MMB_NO'] = df6['UNFY_MMB_NO'].astype(object)

In [144]:
df_c = pd.merge(df2, df6, on ='UNFY_MMB_NO', how ='left')

In [145]:
df_c.groupby('쿠폰사용')['COUPON_CNT'].count()

쿠폰사용
미사용    62
사용     62
Name: COUPON_CNT, dtype: int64

In [85]:
# query run
query7 = '''
        /* 포인트 적립 이력*/ 
        select unfy_mmb_no
        , sum(acml_pint) as acml_amt
        from mmbship.tp_pint_acml
        where coopco_cd = '7050'
        and apv_dt between '20210211' and '20220221'
        and unfy_mmb_no is not null
        group by unfy_mmb_no
        '''
df7 = pd.read_sql(query7, con = conn )
df7['UNFY_MMB_NO'] = df7['UNFY_MMB_NO'].astype(object)

In [86]:
# query run
query8 = '''
        /* 포인트 사용*/
        select unfy_mmb_no
        , sum(use_pint) as use_amt
        from mmbship.tp_pint_use
        where coopco_cd = '7050'
        and use_dt between '20210211' and '20220221'
        and unfy_mmb_no is not null
        group by unfy_mmb_no
        '''
df8 = pd.read_sql(query8, con = conn )
df8['UNFY_MMB_NO'] = df8['UNFY_MMB_NO'].astype(object)

In [87]:
df_d = pd.merge(df2, df7, on ='UNFY_MMB_NO', how ='left')

In [114]:
df_e = pd.merge(df2, df8, on ='UNFY_MMB_NO', how ='left')
df_f = pd.merge(df_e, df7, on ='UNFY_MMB_NO', how ='left').fillna(0)

In [122]:
len(df_f[(df_f['쿠폰사용']=='사용') & (df_f['USE_AMT']>0) & (df_f['ACML_AMT']<=0) ]), len(df_e[(df_e['USE_AMT']>0) ])

(1, 32)

In [123]:
len(df_f[(df_f['쿠폰사용']=='사용') & (df_f['USE_AMT']<=0) & (df_f['ACML_AMT']>0) ]), len(df_d[(df_d['ACML_AMT']>0) ])

(31, 97)

In [124]:
len(df_f[(df_f['쿠폰사용']=='사용') & (df_f['ACML_AMT']>0) & (df_f['USE_AMT']>0) ]), len(df_f[(df_f['ACML_AMT']>0) & (df_f['USE_AMT']>0) ])

(22, 31)

In [92]:
#포인트 사용 : 전체, 쿠폰이용자
df_e.USE_AMT.mean(), df_e[df_e['쿠폰사용']=='사용'].USE_AMT.mean()

(22639.0625, 19147.82608695652)

In [1]:
# df_f[(df_f['쿠폰사용']=='사용') & (df_f['USE_AMT']<=0) & (df_f['ACML_AMT']>0) ]

In [11]:
###############
# 1.데이터 불러오기 : 최근 25개월간, 방문회원  
#####

# query run
query1 = '''with employee_list as 
                (select A.unfy_mmb_no
                 , (case when a.cpn_st_cd = '2' then 'Y' else 'N' end) as gubun
                 from mmbship.TPM_CPN_use B, mmbship.TPM_CPN_ISSU A
                 where A.CPN_ISSU_CD = B.CPN_ISSU_CD(+)
                 and B.can_apv_dt is null
                 and A.cpn_cd in( 'CP220210001' )
                 order by B.reg_DTM)

            --포인트 적립/사용 + 기프트카드사용
            select T.unfy_mmb_no
            , (select code_nm from mmbship.JT_CODE where code_grp_id = 'PM001' and trsc_typ_cd = code) as trsc_typ_cd
            , T.coopco_cd
            , (case when acml_pint = 0 then use_pint else acml_pint end) as use_amt
            , T.tot_sel_amt
            , T.reg_dtm as reg_dtm
            , round((TRUNC(sysdate,'YY') - TO_DATE(btdy, 'YYYYMMDD'))/365,0) as age
            , (substr(I.btdy,1,4)) as AGE_YEAR
            , (case when E.gubun is null then 'T' else E.gubun end) as gubun
            from mmbship.TP_TRSC_PTCL T, employee_list E, tm_mmb_info I
            where T.unfy_mmb_no is not null 
            and T.tot_sel_amt > 0 
            and T.unfy_mmb_no = I.unfy_mmb_no 
            and T.unfy_mmb_no = E.unfy_mmb_no(+)
            and T.reg_dtm between ADD_MONTHS( TRUNC(sysdate,'MM'), -13 ) and ADD_MONTHS( LAST_DAY(sysdate), -2 )
            and T.trsc_typ_cd in ( 'G40', '200', '300')
            order by T.unfy_mmb_no, T.reg_dtm asc
        '''
df2_CJ = pd.read_sql(query1, con = conn )
df2_CJ['UNFY_MMB_NO'] = df2_CJ['UNFY_MMB_NO'].astype(object)

In [13]:
df2_CJ.REG_DTM.min(), df2_CJ.REG_DTM.max()

(Timestamp('2021-02-01 00:03:16'), Timestamp('2022-01-31 16:55:53'))

In [12]:
df = df2_CJ.copy()

In [14]:
today_date = datetime(2022, 2, 1) # We have to set today date in order to calculate Recency metric
rfm = df.groupby('UNFY_MMB_NO').agg({'REG_DTM': lambda date: (today_date - date.max()).days, # Recency calclation
                                     'UNFY_MMB_NO': lambda num: num.count(), # Frequency calculation
                                     'TOT_SEL_AMT': lambda TotalPrice: TotalPrice.sum()}) # Monetary calculation

rfm.columns = ['Recency', 'Frequency', 'Monetary'] # Changing column names

# Converting
rfm['recency_score']   = pd.qcut(rfm['Recency'], 5, [5, 4, 3, 2,1])
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, [1, 2, 3, 4, 5]) 
rfm['monetary_score']  = pd.qcut(rfm['Monetary'], 5, [1, 2, 3, 4, 5]) 

# Calculating RFM score
rfm['RFM_SCORE'] = (rfm['recency_score'].astype('str') + rfm['frequency_score'].astype('str'))

seg_map = {
    r'[1-2][1-2]': '10.hibernating', # For instance means: If F is 1 or 2 and If R is 1 or 2 give hibernating 
    r'[1-2]5': '09.cant_loose',
    r'[1-2][3-4]': '08.at_Risk',
    r'3[1-2]': '07.about_to_sleep',
    r'33': '06.need_attention',
    r'41': '05.promising',
    r'51': '04.new_customers',
    r'[4-5][2-3]': '03.potential_loyalists',
    r'[3-4][4-5]': '02.loyal_customers',
    r'5[4-5]': '01.champions'
}

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [15]:
rfm = rfm.reset_index()

In [16]:
df2 = pd.merge(rfm, df[['UNFY_MMB_NO', 'GUBUN']].drop_duplicates(), on ='UNFY_MMB_NO', how = 'left' )

In [17]:
pd.crosstab(index =df2.segment, columns = df2.GUBUN)

GUBUN,N,T,Y
segment,,,
01.champions,84,8717,54
02.loyal_customers,33,12050,14
03.potential_loyalists,10,8683,6
04.new_customers,4,1323,2
05.promising,4,1927,0
06.need_attention,1,3284,1
07.about_to_sleep,0,4651,3
08.at_Risk,9,8933,9
09.cant_loose,3,1733,1


In [3]:
###############
# 1.데이터 불러오기 : 최근 25개월간, 방문회원  
#####

# query run
query1 = '''with employee_list as 
                (select A.unfy_mmb_no
                 , (case when a.cpn_st_cd = '2' then 'Y' else 'N' end) as gubun
                 from mmbship.TPM_CPN_use B, mmbship.TPM_CPN_ISSU A
                 where A.CPN_ISSU_CD = B.CPN_ISSU_CD(+)
                 and B.can_apv_dt is null
                 and A.cpn_cd in( 'CP220210002' )
                 order by B.reg_DTM)

            --포인트 적립/사용 + 기프트카드사용
            select T.unfy_mmb_no
            , (select code_nm from mmbship.JT_CODE where code_grp_id = 'PM001' and trsc_typ_cd = code) as trsc_typ_cd
            , T.coopco_cd
            , (case when acml_pint = 0 then use_pint else acml_pint end) as use_amt
            , T.tot_sel_amt
            , T.reg_dtm as reg_dtm
            , round((TRUNC(sysdate,'YY') - TO_DATE(btdy, 'YYYYMMDD'))/365,0) as age
            , (substr(I.btdy,1,4)) as AGE_YEAR
            , (case when E.gubun is null then 'T' else E.gubun end) as gubun
            from mmbship.TP_TRSC_PTCL T, employee_list E, tm_mmb_info I
            where T.unfy_mmb_no is not null 
            and T.tot_sel_amt > 0 
            and T.unfy_mmb_no = I.unfy_mmb_no 
            and T.unfy_mmb_no = E.unfy_mmb_no(+)
            and T.reg_dtm between ADD_MONTHS( TRUNC(sysdate,'MM'), -13 ) and ADD_MONTHS( LAST_DAY(sysdate), -2 )
            and T.trsc_typ_cd in ( 'G40', '200', '300')
            order by T.unfy_mmb_no, T.reg_dtm asc
         '''
df2_ILP = pd.read_sql(query1, con = conn )
df2_ILP['UNFY_MMB_NO'] = df2_ILP['UNFY_MMB_NO'].astype(object)

In [5]:
df2_ILP.REG_DTM.min(), df2_ILP.REG_DTM.max()

(Timestamp('2021-02-01 00:03:16'), Timestamp('2022-01-31 16:55:53'))

In [4]:
df = df2_ILP.copy()

In [6]:
today_date = datetime(2022, 2, 1) # We have to set today date in order to calculate Recency metric
rfm = df.groupby('UNFY_MMB_NO').agg({'REG_DTM': lambda date: (today_date - date.max()).days, # Recency calclation
                                     'UNFY_MMB_NO': lambda num: num.count(), # Frequency calculation
                                     'TOT_SEL_AMT': lambda TotalPrice: TotalPrice.sum()}) # Monetary calculation

rfm.columns = ['Recency', 'Frequency', 'Monetary'] # Changing column names

# Converting
rfm['recency_score']   = pd.qcut(rfm['Recency'], 5, [5, 4, 3, 2,1])
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, [1, 2, 3, 4, 5]) 
rfm['monetary_score']  = pd.qcut(rfm['Monetary'], 5, [1, 2, 3, 4, 5]) 

# Calculating RFM score
rfm['RFM_SCORE'] = (rfm['recency_score'].astype('str') + rfm['frequency_score'].astype('str'))

seg_map = {
    r'[1-2][1-2]': '10.hibernating', # For instance means: If F is 1 or 2 and If R is 1 or 2 give hibernating 
    r'[1-2]5': '09.cant_loose',
    r'[1-2][3-4]': '08.at_Risk',
    r'3[1-2]': '07.about_to_sleep',
    r'33': '06.need_attention',
    r'41': '05.promising',
    r'51': '04.new_customers',
    r'[4-5][2-3]': '03.potential_loyalists',
    r'[3-4][4-5]': '02.loyal_customers',
    r'5[4-5]': '01.champions'
}

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [7]:
rfm = rfm.reset_index()

In [8]:
df3 = pd.merge(rfm, df[['UNFY_MMB_NO', 'GUBUN']].drop_duplicates(), on ='UNFY_MMB_NO', how = 'left' )

In [10]:
pd.crosstab(index =df3.segment, columns = df3.GUBUN)

GUBUN,N,T,Y
segment,,,
01.champions,111,8734,10
02.loyal_customers,38,12057,2
03.potential_loyalists,12,8686,1
04.new_customers,5,1324,0
05.promising,3,1928,0
06.need_attention,1,3285,0
07.about_to_sleep,1,4653,0
08.at_Risk,17,8933,1
09.cant_loose,3,1734,0
